## Random forest

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pickle

# Charger les données
file = open('dataset/dataset.pkl', 'rb')
DATA = pickle.load(file)
pheno = DATA['pheno']
X_gpa = DATA['X_gpa']
X_snps = DATA['X_snps']
X_genexp = DATA['X_genexp']

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score
import numpy as np
import pandas as pd

def refine_and_evaluate_model_with_rf(pheno, X_gpa, X_snps, X_genexp, model, param_grid, model_name, importance_threshold=0):
    performances = []
    feature_importance_details = []
    hyperparams_records = []

    # Pour chaque antibiotique
    for antibiotique in pheno.columns[1:]:
        print(f"Traitement de l'antibiotique : {antibiotique}")

        # Extraire la cible (y) et les indices valides
        y = pheno[antibiotique].to_numpy()
        valid_indices = ~np.isnan(y)  
        y = y[valid_indices]

        X_gpa_filtered = X_gpa[valid_indices]
        X_snps_filtered = X_snps[valid_indices]
        X_genexp_filtered = X_genexp[valid_indices]

        # Diviser les données en train/test avec stratification
        X_gpa_train, X_gpa_test, X_snps_train, X_snps_test, X_genexp_train, X_genexp_test, y_train, y_test = train_test_split(
            X_gpa_filtered, X_snps_filtered, X_genexp_filtered, y, 
            test_size=0.2, random_state=42, stratify=y
        )

        scaler_genexp = StandardScaler()
        X_genexp_train = scaler_genexp.fit_transform(X_genexp_train)  # Fit + transform sur l'entraînement
        X_genexp_test = scaler_genexp.transform(X_genexp_test)        # Transform uniquement sur le test

        X_train = np.hstack([X_gpa_train, X_snps_train, X_genexp_train])
        X_test = np.hstack([X_gpa_test, X_snps_test, X_genexp_test])

        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='recall', verbose=1)
        grid_search.fit(X_train, y_train)

        # Meilleur modèle trouvé
        meilleur_modele = grid_search.best_estimator_

        # Calculer l'importance des caractéristiques
        feature_importances = meilleur_modele.feature_importances_

        # Ajouter les détails de l'importance des caractéristiques
        for idx, importance in enumerate(feature_importances):
            feature_importance_details.append({
                'Antibiotique': antibiotique,
                'Feature_Index': idx,
                'Importance': importance
            })

        # Filtrer les colonnes par importance
        important_features = feature_importances > importance_threshold

        # Identifier les colonnes filtrées pour GPA, SNPs et GENEXP
        colonnes_support = np.arange(X_train.shape[1])
        colonnes_gpa = colonnes_support[:X_gpa_train.shape[1]][important_features[:X_gpa_train.shape[1]]]
        colonnes_snps = colonnes_support[X_gpa_train.shape[1]:X_gpa_train.shape[1] + X_snps_train.shape[1]][
            important_features[X_gpa_train.shape[1]:X_gpa_train.shape[1] + X_snps_train.shape[1]]
        ]
        colonnes_genexp = colonnes_support[X_gpa_train.shape[1] + X_snps_train.shape[1]:][
            important_features[X_gpa_train.shape[1] + X_snps_train.shape[1]:]
        ]

        # Prédictions et calcul du rappel macro
        y_pred = meilleur_modele.predict(X_test)
        rappel_macro = recall_score(y_test, y_pred, average='macro')
        print(f"Rappel macro pour {antibiotique} avec {model_name}: {rappel_macro:.4f}")

        # Stocker les performances pour chaque antibiotique
        performances.append({
            'Antibiotique': antibiotique,
            'Recall': rappel_macro,
            'Colonnes_GPA': colonnes_gpa.tolist(),
            'Colonnes_SNPs': colonnes_snps.tolist(),
            'Colonnes_Expression_Genetique': colonnes_genexp.tolist()
        })

        hyperparams_records.append({
            'Antibiotique': antibiotique,
            'Meilleur_paramètre': grid_search.best_params_,
            'Valeurs_des_hyperparamètres': param_grid
        })

    performances_df = pd.DataFrame(performances)
    hyperparams_df = pd.DataFrame(hyperparams_records)

    # fichier CSV
    csv_filename = f'result_{model_name}.csv'
    performances_df.to_csv(csv_filename, index=False, sep=';')
    print(f"Les performances ont été enregistrées dans {csv_filename}")

    feature_importance_df = pd.DataFrame(feature_importance_details)

    importance_csv_filename = f'feature_importance_{model_name}.csv'
    feature_importance_df.to_csv(importance_csv_filename, index=False, sep=';')
    print(f"Les importances des caractéristiques ont été enregistrées dans {importance_csv_filename}")

    hyperparams_csv_filename = f'hyperparametres_{model_name}.csv'
    hyperparams_df.to_csv(hyperparams_csv_filename, index=False, sep=';')
    print(f"Les hyperparamètres ont été enregistrés dans {hyperparams_csv_filename}")

    return performances_df, feature_importance_df

In [3]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
}

performances_df, feature_importance_df = refine_and_evaluate_model_with_rf(pheno, X_gpa, X_snps, X_genexp, model, param_grid, "RandomForest")

print("\nPerformances du modèle après sélection des caractéristiques:")
print(performances_df)

print("\nImportances des caractéristiques:")
print(feature_importance_df)

Traitement de l'antibiotique : Tobramycin
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Rappel macro pour Tobramycin avec RandomForest: 0.9066
Traitement de l'antibiotique : Ceftazidim
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Rappel macro pour Ceftazidim avec RandomForest: 0.6114
Traitement de l'antibiotique : Ciprofloxacin
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Rappel macro pour Ciprofloxacin avec RandomForest: 0.7875
Traitement de l'antibiotique : Meropenem
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Rappel macro pour Meropenem avec RandomForest: 0.5932
Traitement de l'antibiotique : Colistin
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Rappel macro pour Colistin avec RandomForest: 0.6243
Les performances ont été enregistrées dans result_RandomForest.csv
Les importances des caractéristiques ont été enregistrées dans feature_importance_RandomForest.csv
Les hyperparamètres ont été enregistrés dans